In [1]:
import concurrent.futures
import multiprocessing
import time
from typing import Dict
import sys

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

sys.path.append("..")

from src.game.yahtzee_game import YahtzeeGame
from src.simulation.simulator import Simulator
from src.strategies.base_strategy import BaseStrategy
from src.strategies.random_strategy import RandomStrategy
from src.strategies.rule_based_strategy import RuleBasedStrategy
from src.strategies.expected_value_strategy import ExpectedValueStrategy
from src.strategies.expected_value_v2_strategy import ExpectedValueV2Strategy

In [2]:
def run_parallel_simulations(
    num_games: int, strategies: Dict[str, BaseStrategy], num_workers: int = None, use_parallel: bool = True
):
    """
    Run multiple Yahtzee game simulations in parallel or sequentially

    Args:
        num_games: Total number of games to simulate
        num_workers: Number of parallel workers (defaults to CPU count if None)
        use_parallel: Whether to use parallel processing

    Returns:
        Combined dictionary of win counts by player
    """
    # Use CPU count if num_workers is not specified
    if num_workers is None and use_parallel:

        num_workers = multiprocessing.cpu_count()

    start_time = time.time()

    if use_parallel:
        # Parallel execution
        games_per_worker = num_games // num_workers

        # Define the task for each worker
        def simulation_task():
            simulator = Simulator()
            return simulator.run_game_simulation(games_per_worker, strategies)

        # Run simulations in parallel
        combined_results = {player: 0 for player in strategies}
        with concurrent.futures.ProcessPoolExecutor(
            max_workers=num_workers
        ) as executor:
            futures = [executor.submit(simulation_task) for _ in range(num_workers)]
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                for player, wins in result.items():
                    combined_results[player] += wins
    else:
        # Sequential execution
        simulator = Simulator()
        combined_results = simulator.run_game_simulation(num_games, strategies)

    end_time = time.time()
    print(f"Simulation completed in {end_time - start_time:.2f} seconds")
    print(
        f"Mode: {'Parallel' if use_parallel else 'Sequential'}, Workers: {num_workers if use_parallel else 1}"
    )

    return combined_results

In [3]:
def collect_detailed_stats(
    num_games: int, strategies: Dict[str, BaseStrategy]
) -> pd.DataFrame:
    all_game_stats = []

    for game_num in range(num_games):
        game = YahtzeeGame()

        # Add players to the game
        for player_name in strategies:
            game.add_player(player_name)

        # Create dictionary to track scores
        player_scores = {player_name: [] for player_name in strategies}

        # Play through each turn manually to collect data
        for turn in range(13):  # Yahtzee has 13 turns
            for i, player in enumerate(game.players):
                player_name = (
                    player.name
                )  # Assuming Player objects have a name attribute
                initial_score = player.scorecard.get_total_score()

                # Use strategy to make moves
                strategy = strategies[player_name]
                game.play_turn(strategy)

                # Record score change
                new_score = player.scorecard.get_total_score()
                player_scores[player_name].append(new_score - initial_score)

        winner = game.determine_winner()
        final_scores = {
            player.name: player.scorecard.get_total_score() for player in game.players
        }

        for player_name in strategies:
            game_data = {
                "game": game_num,
                "player": player_name,
                "strategy": player_name,
                "final_score": final_scores[player_name],
                "winner": player_name == winner,
                "score_progression": player_scores[player_name],
            }
            all_game_stats.append(game_data)

    return pd.DataFrame(all_game_stats)

In [4]:
# Compare strategies across different categories
def analyze_category_performance(
    num_games: int, strategies: Dict[str, BaseStrategy]
) -> pd.DataFrame:
    category_stats = []

    for _ in range(num_games):
        for strategy_name, strategy in strategies.items():
            simulator = Simulator()
            game = YahtzeeGame()
            game.add_player("Player")

            # Play full game with this strategy
            game.play_game({"Player": strategy})

            # Extract category scores
            scorecard = game.players[0].scorecard  # Access the first player
            for category, score in scorecard.scores.items():
                if score is not None:  # Only include categories that were used
                    category_stats.append(
                        {
                            "strategy": strategy_name,
                            "category": category.value,
                            "score": score,
                        }
                    )

    return pd.DataFrame(category_stats)

In [5]:
# Run 1000 games with 4 parallel workers
N_SIMULATIONS = 100
USE_PARALLEL = False  # Set to True for parallel execution

STRATEGIES = {
    "Rule-Based": RuleBasedStrategy(),
    "Random": RandomStrategy(),
    "Expected Value": ExpectedValueStrategy(),
    "Expected Value V2": ExpectedValueV2Strategy(),
}

win_counts = run_parallel_simulations(
    N_SIMULATIONS, STRATEGIES, use_parallel=USE_PARALLEL
)

# Visualize win distribution
fig = px.pie(
    values=list(win_counts.values()),
    names=list(win_counts.keys()),
    title="Win Distribution by Strategy",
    color_discrete_sequence=px.colors.qualitative.Set3,
)
fig.show()

# Collect detailed stats for a smaller number of games

detailed_stats = collect_detailed_stats(N_SIMULATIONS, STRATEGIES)

# Visualize score distribution by strategy
fig = px.box(
    detailed_stats,
    x="strategy",
    y="final_score",
    color="strategy",
    title="Distribution of Final Scores by Strategy",
    points="all",
)
fig.show()

# Visualize score progression
avg_progression = (
    detailed_stats.groupby(["strategy"])
    .apply(
        lambda x: pd.DataFrame(
            {
                turn: [scores[turn] for scores in x["score_progression"]]
                for turn in range(13)
            }
        ).mean()
    )
    .reset_index()
)

# Reshape for plotting
progression_data = []
for _, row in avg_progression.iterrows():
    strategy = row["strategy"]
    for turn in range(13):
        progression_data.append(
            {"strategy": strategy, "turn": turn + 1, "avg_score": row[turn]}
        )

progression_df = pd.DataFrame(progression_data)

# Create line plot
fig = px.line(
    progression_df,
    x="turn",
    y="avg_score",
    color="strategy",
    title="Average Points Scored Per Turn",
    labels={"turn": "Turn Number", "avg_score": "Average Points"},
)
fig.show()

# Run analysis
category_performance = analyze_category_performance(N_SIMULATIONS, STRATEGIES)

# Visualize category performance
fig = px.box(
    category_performance,
    x="category",
    y="score",
    color="strategy",
    facet_col="strategy",
    title="Score Distribution by Category and Strategy",
)
fig.update_xaxes(tickangle=45)
fig.show()

Initializing lookup tables for ExpectedValueStrategy...
Computing third roll values...
Computing second roll values...
Computing first roll values...
Lookup tables initialized!


KeyboardInterrupt: 